In [1]:
import gurobipy as gp

#Abrindo a intancia
def build_instance(file_name):
    print(f"BUILDING INSTANCE {file_name.split("/")[len(file_name.split("/"))-1]}")
    instance_file = open(file_name,"r")
    
    lines = [line.strip() for line in instance_file.readlines()]
    current_line = 0
    
    #Salvando quantidade de clientes e quantidade e lockers
    qty_customer = int(lines[current_line].split(" ")[0])
    qty_locker = int(lines[current_line].split(" ")[1])
    
    current_line += 1
    
    max_vehicle = int(lines[current_line].split(" ")[0])
    vehicle_capacity = float(lines[current_line].split(" ")[1])
    
    current_line += 1
    
    demands = list()
    ## Salvando demands de clients
    for demand in range(current_line,qty_customer+2):
        demands.append(float(lines[demand]))
    
    current_line += qty_customer
    
    #Preenchendo nos
    nodes = dict()
    
    #Deposito
    depot = {
        'label': "D",
        'x' : float(lines[current_line].split(" ")[0]),
        'y' : float(lines[current_line].split(" ")[1]),
        'earliest' : float(lines[current_line].split(" ")[2]),
        'latest' : float(lines[current_line].split(" ")[3]),
        'service_time' : float(lines[current_line].split(" ")[4]),
        'demand': 0,
        'type' : int(lines[current_line].split(" ")[5])
    }
    nodes['D']=depot
    current_line += 1

    #Clientes
    customers = dict()
    customer_count = 1
    for i in range(current_line, current_line + qty_customer):
        customer = {
        'label' : f"C{customer_count}",
        'x' : float(lines[i].split(" ")[0]),
        'y' : float(lines[i].split(" ")[1]),
        'earliest' : float(lines[i].split(" ")[2]),
        'latest' : float(lines[current_line].split(" ")[3]),
        'service_time' : float(lines[i].split(" ")[4]),
        'demand': demands[customer_count-1],
        'type' : int(lines[i].split(" ")[5])
        }
        customer_count+=1
        customers[customer['label']]=customer
        nodes[customer['label']]=customer
    current_line += qty_customer

    #Lockers
    lockers = dict()
    locker_count = 1
    for i in range(current_line,current_line+qty_locker):
        locker = {
        'label' : f"P{locker_count}",
        'x' : float(lines[i].split(" ")[0]),
        'y' : float(lines[i].split(" ")[1]),
        'earliest' : float(lines[i].split(" ")[2]),
        'latest' : float(lines[current_line].split(" ")[3]),
        'service_time' : float(lines[i].split(" ")[4]),
        'demand': 0,
        'customers': list(),
        'type' : int(lines[i].split(" ")[5])
        }
        locker_count+=1
        lockers[locker['label']] = locker
        nodes[locker['label']]=locker
    current_line += qty_locker

    #Atribuição de lockers
    current_customer = 0
    for i in range(current_line, current_line+qty_customer):
        try:
            customer = customers[list(customers.keys())[current_customer]]
            
            locker_index = lines[i].split(" ").index("1")
            locker = lockers[list(lockers.keys())[locker_index]]
            customer['locker'] = locker['label']
            locker['customers'].append(customer['label'])
        except ValueError:    
            customer['locker'] = ""
        current_customer += 1
    current_line += qty_customer
    
    #Matriz de Distancia
    costs = dict()
    for i,node_i in enumerate(nodes):
         current_node_i = nodes[node_i]
         current_node_i['index'] = i
         for j,node_j in enumerate(nodes):
             current_node_j = nodes[node_j]
             distance = ((current_node_i['x']-current_node_j['x'])**2 + (current_node_i['y']-current_node_j['y'])**2)**(1/2)
             costs[current_node_i['label'],current_node_j['label']] = distance
    print(f"Qty customer:{qty_customer}")
    print(f"Qty locker:{qty_locker}")
    print(f"Max vehicles:{max_vehicle}")
    print(f"Vehicle capacity:{vehicle_capacity}")
    print(f"BUILD FINISHED")
    print(sum(demands))
    return {
        'qty_customer': qty_customer,
        'qty_locker': qty_locker,
        'qty_nodes': qty_locker,
        'total_demands': sum(demands),
        'min_vehicles': round(sum(demands)/vehicle_capacity),
        'max_vehicle': max_vehicle,
        'vehicle_capacity': vehicle_capacity,
        'nodes': nodes,
        'customers': customers,
        'lockers': lockers,
        'depot': depot,
        'costs': costs
    }

def print_labels(nodes, lockers, customers, customers_hc, customers_lc, customers_hlc, vehicles):
    print(f"Nodes: {nodes}")
    print(f"Depot: {depot}")
    print(f"Lockers: {lockers}")
    print(f"Customers: {customers}")
    print(f"Customers HC : {customers_hc}")
    print(f"Customers LC: {customers_lc}")
    print(f"Customers HLC: {customers_hlc}")
    print(f"Vehicles: {vehicles}")
    
# instance = build_instance("../../instances/vrppl/inst_test")
instance = build_instance("../../instances/vrppl/25/R101_co_25.txt")

BUILDING INSTANCE R101_co_25.txt
Qty customer:25
Qty locker:2
Max vehicles:25
Vehicle capacity:200.0
BUILD FINISHED
332.0


In [2]:
qty_vehicles = 3
v_capacity = int(instance['vehicle_capacity'])
#Rótulos
nodes = list(instance['nodes'].keys())
lockers = [node for node in nodes if instance['nodes'][node]['type'] == 4]
customers = list(instance['customers'].keys())
customers_hc = [customer for customer in customers if instance['nodes'][customer]['type'] == 1]
customers_lc = [customer for customer in customers if instance['nodes'][customer]['type'] == 2]
customers_hlc = [customer for customer in customers if instance['nodes'][customer]['type'] == 3]
depot = [node for node in nodes if instance['nodes'][node]['type'] == 0][0]


# print(edges)

#Veiculos
vehicles = list()
for i in range(qty_vehicles):
    vehicles.append(f"v_{i}")

print_labels(nodes, lockers, customers, customers_hc, customers_lc, customers_hlc, vehicles)


Nodes: ['D', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'P1', 'P2']
Depot: D
Lockers: ['P1', 'P2']
Customers: ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25']
Customers HC : ['C1', 'C10', 'C15', 'C17', 'C19']
Customers LC: ['C3', 'C6', 'C7', 'C8', 'C13', 'C14', 'C24']
Customers HLC: ['C2', 'C4', 'C5', 'C9', 'C11', 'C12', 'C16', 'C18', 'C20', 'C21', 'C22', 'C23', 'C25']
Vehicles: ['v_0', 'v_1', 'v_2']


In [3]:
#Criando dicts

#Nos
nodes_dict = instance['nodes']

#Custos
costs = instance['costs']

#Demandas, tempo de serviços e janelas de tempo
demands = dict()
service_time = dict()
tw_a = dict()
tw_b = dict()
for node in nodes_dict:
    demands[node] = nodes_dict[node]['demand']
    service_time[node] = nodes_dict[node]['service_time']
    tw_a[node] = nodes_dict[node]['earliest']
    tw_b[node] = nodes_dict[node]['latest']
    
#Atribuições de locker
locker_attrib = dict()

for locker in lockers:
    for customer in customers:
        if(customer in nodes_dict[locker]['customers']):
            locker_attrib[customer,locker] = 1
        else:
            locker_attrib[customer,locker] = 0


In [4]:
#Criando modelo
m = gp.Model()
m.setParam(gp.GRB.Param.OutputFlag,0)

#Criando Variaveis
x_ijk = m.addVars(nodes,nodes, vehicles, vtype=gp.GRB.BINARY)
h_i = m.addVars(customers, vtype = gp.GRB.BINARY)
l_i = m.addVars(customers, vtype = gp.GRB.BINARY)
y_ij = m.addVars(customers,lockers, vtype = gp.GRB.BINARY)
psi_jk = m.addVars(lockers,vehicles, vtype = gp.GRB.INTEGER)
delta_k = m.addVars(vehicles, vtype = gp.GRB.CONTINUOUS)
mu_ik = m.addVars(nodes,vehicles, vtype = gp.GRB.CONTINUOUS)
vl_k = m.addVars(vehicles, vtype = gp.GRB.CONTINUOUS)


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2621941
Academic license 2621941 - for non-commercial use only - registered to ra___@gmail.com


In [5]:
M = 1000000

#Função objetivo
m.setObjective(
    gp.quicksum(costs[node_i,node_j] * x_ijk[node_i,node_j,v] for node_i in nodes for node_j in nodes for v in vehicles)
)
#Restrições

# #Cliente hc apenas em casa
c2 = m.addConstrs(
    gp.quicksum(x_ijk[customer_i,node_j,v] for node_j in nodes for v in vehicles) == 1 for customer_i in customers_hc
)

#Não visitar cliente lc
c3 = m.addConstrs(
    gp.quicksum(x_ijk[customer_i,node_j,v] for node_j in nodes for v in vehicles) <= h_i[customer_i] for customer_i in customers_hlc
)

#Não visitar cliente se for customer_lc
c4 = m.addConstrs(
    gp.quicksum(x_ijk[customer_i,node_j,v] for node_j in nodes for v in vehicles) == 0 for customer_i in customers_lc
)
# print(v_k)
#Sempre sair do deposito
c5 = m.addConstrs(
    gp.quicksum(x_ijk[depot,customer_j,v] for customer_j in customers) <= 1 for v in vehicles
)

#Dinâmica de fluxo
c6 = m.addConstrs(
    gp.quicksum(x_ijk[node_i,node_j,v] for node_i in nodes if node_i != node_j) ==
    gp.quicksum(x_ijk[node_j,node_i,v] for node_i in nodes if node_j != node_i) for node_j in nodes for v in vehicles  
    
)

#Sempre chiegar no deposito
c7 = m.addConstrs(
    gp.quicksum(x_ijk[customer_i,depot,v] for customer_i in customers) <= 1 for v in vehicles
)


#Limitação de capacidade do locker
c8 =  m.addConstrs(
    gp.quicksum(demands[customer_j] * x_ijk[node_i,customer_j,v] for node_i in nodes for customer_j in customers if node_i != customer_j)
    + gp.quicksum(psi_jk[locker_m,v] * x_ijk[node_p,locker_m,v]  for node_p in nodes for locker_m in lockers if node_p != locker_m)
    <= v_capacity
    for v in vehicles
)

#Tempo pra sair do deposito
c9 =  m.addConstrs(
    delta_k[v_k] + costs[depot,node_i] - mu_ik[node_i,v_k] <= M*(1-x_ijk[depot,node_i,v_k])
    for v_k in vehicles for node_i in customers + lockers
)

#Tempo entre nós
c10 =  m.addConstrs(
    mu_ik[node_i,v_k] + service_time[node_i] + costs[node_i,node_j] - mu_ik[node_j,v_k] <= M*(1-x_ijk[node_i,node_j,v_k])
    for v_k in vehicles for node_i in customers + lockers for node_j in customers + lockers
)

#Tempo para o ultimo no
c11 =  m.addConstrs(
    mu_ik[node_i,v_k] - vl_k[v_k] <= M*(1-x_ijk[node_i,depot,v_k])
    for v_k in vehicles for node_i in customers + lockers
)

#Time window
c12a =  m.addConstrs(
    gp.quicksum(mu_ik[customer_i,v_k] * h_i[customer_i] for v_k in vehicles )
    <= tw_b[customer_i] * h_i[customer_i]
    for customer_i in customers
)

c12b =  m.addConstrs(
    tw_a[customer_i] * h_i[customer_i] <=
    gp.quicksum(mu_ik[customer_i,v_k] * h_i[customer_i] for v_k in vehicles )
    for customer_i in customers
)


# --- #

#Ou casa ou locker vai atender o N_HLC
c13 =  m.addConstrs(
    h_i[customer_i] + l_i[customer_i] == 1 for customer_i in customers
)

#h_i sempre 1 pra cliente hc
c14 =  m.addConstrs(
    h_i[customer_i] == 1 for customer_i in customers_hc
)

#l_i sempre 1 pra cliente lc
c15 =  m.addConstrs(
    l_i[customer_i] == 1 for customer_i in customers_lc
)

# \sum y_ij = l_i 
c16 =  m.addConstrs(
    gp.quicksum(y_ij[customer_i,locker_j] for locker_j in lockers) == l_i[customer_i] for customer_i in customers
)

#Atribuição de lockers, atribui o locker certo
c17 =  m.addConstrs(
    y_ij[customer_i,locker_j] <= locker_attrib[customer_i,locker_j] for customer_i in customers for locker_j in lockers
)

# Garante que a quantidade de items deixados no locker seja igual a quantidade de loads cujo l_i == 1
c18 =  m.addConstrs(
    (gp.quicksum(psi_jk[locker_j, v_k] for v_k in vehicles) == gp.quicksum(y_ij[customer_i,locker_j] * demands[customer_i] for customer_i in customers_lc + customers_hlc))
    for locker_j in lockers
)

#---#
# Garante que o locker seja visitadoe seja visitado se l_j = 1
c20 =  m.addConstrs(
    gp.quicksum(x_ijk[node_i,locker_j,v] for node_i in nodes) * M >= psi_jk[locker_j,v]
    for locker_j in lockers for v in vehicles
)

# Garante que o cliente seja visitado se h_j = 1
c21 =  m.addConstrs(
    gp.quicksum(x_ijk[node_i,customer_j,v] for node_i in nodes for v in vehicles) >= h_i[customer_j] for customer_j in customers
)
m.optimize()

In [20]:

print(f"Resultado final: {m.objVal}")

for vehicle in vehicles:
    print(vehicle,": ", end = " ")
    for node_i in nodes:
        for node_j in nodes:
            if round(x_ijk[node_i,node_j,vehicle].X) == 1:
                print(f"{node_i},{node_j}", " ", end="")
                
    print('\n')

print("\n\nVariavel h_i")
for i in customers:
    print(f"h_{i}= {round(h_i[i].X)}", end=" ")
print("\n\nVariavel l_i")
for i in customers:
    print(f"l_{i}= {round(l_i[i].X)}", end=" ")
print("\n\nVariavel y_ij")
for i in customers:
    for j in lockers:
        if (round(y_ij[i,j].X) == 1):
            print(f"{i}{j}", end=" ")
print("\n\nVariavel psi_jk")
for j in lockers:
    for k in vehicles:
        print(f"psi_{j}{k}= {round(psi_jk[j,k].X)}", end=" ")

print("\n\nVariavel delta_k")
for k in vehicles:
    print(f"delta_{k}= {round(delta_k[k].X)}", end=" ")

print("\n\nVariavel vl_k")
for k in vehicles:
    print(f"vl_{k}= {vl_k[k].X}", end=" ")

print("\n",demands)

Resultado final: 178.84376024636876
v_0 :  D,C15  C15,P2  C17,D  P2,C17  

v_1 :  

v_2 :  D,C1  C1,C10  C10,P1  C19,D  P1,C19  



Variavel h_i
h_C1= 1 h_C2= 0 h_C3= 0 h_C4= 0 h_C5= 0 h_C6= 0 h_C7= 0 h_C8= 0 h_C9= 0 h_C10= 1 h_C11= 0 h_C12= 0 h_C13= 0 h_C14= 0 h_C15= 1 h_C16= 0 h_C17= 1 h_C18= 0 h_C19= 1 h_C20= 0 h_C21= 0 h_C22= 0 h_C23= 0 h_C24= 0 h_C25= 0 

Variavel l_i
l_C1= 0 l_C2= 1 l_C3= 1 l_C4= 1 l_C5= 1 l_C6= 1 l_C7= 1 l_C8= 1 l_C9= 1 l_C10= 0 l_C11= 1 l_C12= 1 l_C13= 1 l_C14= 1 l_C15= 0 l_C16= 1 l_C17= 0 l_C18= 1 l_C19= 0 l_C20= 1 l_C21= 1 l_C22= 1 l_C23= 1 l_C24= 1 l_C25= 1 

Variavel y_ij
C2P2 C3P1 C4P2 C5P2 C6P2 C7P1 C8P1 C9P1 C11P1 C12P1 C13P2 C14P2 C16P2 C18P1 C20P1 C21P2 C22P2 C23P2 C24P1 C25P2 

Variavel psi_jk
psi_P1v_0= 0 psi_P1v_1= 0 psi_P1v_2= 98 psi_P2v_0= 181 psi_P2v_1= 0 psi_P2v_2= 0 

Variavel delta_k
delta_v_0= 0 delta_v_1= 0 delta_v_2= 0 

Variavel vl_k
vl_v_0= 171.0 vl_v_1= 0.0 vl_v_2= 171.0 
 {'D': 0, 'C1': 10.0, 'C2': 7.0, 'C3': 13.0, 'C4': 19.0, 'C5': 26.

In [7]:
for var in x_ijk:
    if round(x_ijk[var].X) == 1:
        print(var, round(x_ijk[var].X))

('D', 'C1', 'v_2') 1
('D', 'C15', 'v_0') 1
('C1', 'C10', 'v_2') 1
('C10', 'P1', 'v_2') 1
('C15', 'P2', 'v_0') 1
('C17', 'D', 'v_0') 1
('C19', 'D', 'v_2') 1
('P1', 'C19', 'v_2') 1
('P2', 'C17', 'v_0') 1
